### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 
csvfile = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csvfile)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


## Player Count

* Display the total number of players


In [4]:
player_count = purchase_data.SN.nunique()
summarytable1 = pd.DataFrame({"Total Players": [player_count]})
summarytable1

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
# purchase_data[["Item ID","Price"]].groupby("Item ID").mean()
unique_items = purchase_data["Item ID"].nunique()
avg_price = round(purchase_data.Price.mean(),2)
purchase_count = len(purchase_data)
total_revenue = round(purchase_data.Price.sum(),2)

summarytable2 = pd.DataFrame()    
summarytable2["Number of Unique Items"] = [unique_items]
summarytable2["Average Price"] = [avg_price]
summarytable2["Number of Purchases"] = [purchase_count]
summarytable2["Total Revenue"] = [total_revenue]
summarytable2


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
#Gender Demographics - technique 1
female_mask = purchase_data.Gender == "Female"
male_mask = purchase_data.Gender == "Male"
other_mask = purchase_data.Gender == "Other / Non-Disclosed"
female_count = len(pd.unique(purchase_data[female_mask].SN))
male_count = len(pd.unique(purchase_data[male_mask].SN))
other_count = len(pd.unique(purchase_data[other_mask].SN))

summarytable3 = pd.DataFrame({"Counts": [female_count, male_count, other_count], 
                              "Percent of Player": [round(female_count/len(purchase_data.SN.value_counts())*100,2), 
                                                    round(male_count/len(purchase_data.SN.value_counts())*100,ndigits=2),
                                                    round(other_count/len(purchase_data.SN.value_counts())*100,ndigits=2)]},
                             index=['Female','Male','Other/Non-Disclosed'])
summarytable3

,Counts,Percent of Player
Female,81,14.06
Male,484,84.03
Other/Non-Disclosed,11,1.91


In [20]:
#Gender Demographics - technique 2
gender_SN = purchase_data.loc[:,["SN","Age","Gender"]].drop_duplicates().reset_index(drop=True)
gender_total = pd.DataFrame(gender_SN.groupby("Gender").size(),columns=["Counts"])
gender_total["Percentage"]= gender_total["Counts"] / len(gender_SN) * 100
gender_total

,Counts,Percentage
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
#Purchase Analysis - technique 1
purch_count_f = len(purchase_data[female_mask])
purch_count_m = len(purchase_data[male_mask])
purch_count_o = len(purchase_data[other_mask])

purchase_count_df = pd.DataFrame()
purchase_count_df["Gender"] = ["Female","Male","Other / Non-Disclosed"]
purchase_count_df["Purchase Count"] = [purch_count_f,purch_count_m,purch_count_o]

avg_price_gender = pd.DataFrame(round(purchase_data.groupby("Gender").Price.mean(),2))
total_price = round(purchase_data.groupby("Gender").Price.sum(),2)

avg_total_f = purchase_data[female_mask].Price.sum()/female_count
avg_total_m = purchase_data[male_mask].Price.sum()/male_count
avg_total_o = purchase_data[other_mask].Price.sum()/other_count

avg_total_df = pd.DataFrame()
avg_total_df["Gender"] = ["Female","Male","Other / Non-Disclosed"]
avg_total_df["Avg Total Purchase per Person"] = [avg_total_f,avg_total_m,avg_total_o]

mergetempdf = pd.merge(purchase_count_df,avg_price_gender,on="Gender")
mergetempdf2 = pd.merge(mergetempdf,total_price, on="Gender")
mergetempdf3 = pd.merge(mergetempdf2,avg_total_df, on="Gender")
summarytable4 = mergetempdf3.rename(columns={"Price_x":"Average Purchase Price","Price_y":"Total Purchase Value"})
summarytable4

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,3.20,361.94,4.468395
1,Male,652,3.02,1967.64,4.065372
2,Other / Non-Disclosed,15,3.35,50.19,4.562727


In [30]:
#Purchase Analysis - technique 2
avg_price_gender = pd.DataFrame(round(purchase_data.groupby("Gender").Price.mean(),2))
total_price = round(purchase_data.groupby("Gender").Price.sum(),2)

x = pd.merge(gender_total,avg_price_gender,on="Gender")
summary4 = pd.merge(x,total_price,on="Gender")
summary4['Avg Total Purchase per Person'] = total_price/gender_total["Counts"]

summary4.rename(columns = {'Price_x':'Avg Purchase Price','Price_y':'Total Purchase Value'})

,Counts,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,81,3.20,361.94,4.468395
Male,484,3.02,1967.64,4.065372
Other / Non-Disclosed,11,3.35,50.19,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
bins = [0,10,20,30,40,50]
groups = ["<=10","11-20","21-30","31-40","41+"]
purchase_data["Age_Group"] = pd.cut(purchase_data["Age"], bins, labels=groups, include_lowest=True)
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age_Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,11-20
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,31-40
2,2,Ithergue48,24,Male,92,Final Critic,4.88,21-30
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,21-30
4,4,Iskosia90,23,Male,131,Fury,1.44,21-30
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,21-30
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,21-30
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,11-20
778,778,Sisur91,7,Male,92,Final Critic,4.19,<=10


In [11]:
to10mask = purchase_data.Age_Group == "<=10"
to20mask = purchase_data.Age_Group == "11-20"
to30mask = purchase_data.Age_Group == "21-30"
to40mask = purchase_data.Age_Group == "31-40"
to50mask = purchase_data.Age_Group == "41+"


to10_SN = len(pd.unique(purchase_data[to10mask].SN))
to20_SN = len(pd.unique(purchase_data[to20mask].SN))
to30_SN = len(pd.unique(purchase_data[to30mask].SN))
to40_SN = len(pd.unique(purchase_data[to40mask].SN))
to50_SN = len(pd.unique(purchase_data[to50mask].SN))

to10_per = round((to10_SN/player_count)*100,2)
to20_per = round((to20_SN/player_count)*100,2)
to30_per = round((to30_SN/player_count)*100,2)
to40_per = round((to40_SN/player_count)*100,2)
to50_per = round((to50_SN/player_count)*100,2)

summarytable5 = pd.DataFrame()

summarytable5["Age Groups"] = groups
summarytable5["Total Count"] = [to10_SN,to20_SN,to30_SN,to40_SN,to50_SN]
summarytable5["Percentage of Players"] = [to10_per,to20_per,to30_per,to40_per,to50_per]
summarytable5

,Age Groups,Total Count,Percentage of Players
0,<=10,24,4.17
1,11-20,191,33.16
2,21-30,291,50.52
3,31-40,63,10.94
4,41+,7,1.22


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
to10_purchases = len(purchase_data[to10mask])
to20_purchases = len(purchase_data[to20mask])
to30_purchases = len(purchase_data[to30mask])
to40_purchases = len(purchase_data[to40mask])
to50_purchases = len(purchase_data[to50mask])

tempdf = pd.DataFrame()
tempdf["Age_Group"] = groups
tempdf["Purchase Count"] = [to10_purchases,to20_purchases,to30_purchases,to40_purchases,to50_purchases]

avg_price_ages = pd.DataFrame(round(purchase_data.groupby("Age_Group").Price.mean(),2))
total_price_ages = round(purchase_data.groupby("Age_Group").Price.sum(),2)

avg_total_10 = purchase_data[to10mask].Price.sum()/to10_SN
avg_total_20 = purchase_data[to20mask].Price.sum()/to20_SN
avg_total_30 = purchase_data[to30mask].Price.sum()/to30_SN
avg_total_40 = purchase_data[to40mask].Price.sum()/to40_SN
avg_total_50 = purchase_data[to50mask].Price.sum()/to50_SN

avg_total_age = pd.DataFrame()
avg_total_age["Age_Group"] = groups
avg_total_age["Avg Total Purchase per Person"] = [avg_total_10,avg_total_20,avg_total_30,avg_total_40,avg_total_50]

mergetemp1 = pd.merge(tempdf,avg_price_ages,on="Age_Group")
mergetemp2 = pd.merge(mergetemp1,total_price_ages, on="Age_Group")
mergetemp3 = pd.merge(mergetemp2,avg_total_age, on="Age_Group")
summarytable6 = mergetemp3.rename(columns={"Price_x":"Average Purchase Price","Price_y":"Total Purchase Value"})
summarytable6

,Age_Group,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<=10,32,3.40,108.96,4.540000
1,11-20,254,3.06,778.16,4.074136
2,21-30,402,2.99,1203.06,4.134227
3,31-40,85,3.15,268.06,4.254921
4,41+,7,3.08,21.53,3.075714


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
groupby_SN = purchase_data.groupby(["SN"])
SN_sum = groupby_SN.Price.sum()
SN_avg =  groupby_SN.Price.mean()
SN_count = groupby_SN["Purchase ID"].count()

temp1 = pd.merge(SN_sum,SN_avg,on="SN")
temp2 = pd.merge(temp1,SN_count,on="SN")
temp3 = temp2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Average Purchase Value", "Purchase ID": "Purchase Count"})
summarytable7 = temp3.sort_values("Total Purchase Value", ascending=False)
summarytable7.head()

,Total Purchase Value,Average Purchase Value,Purchase Count
SN,,,
Lisosia93,18.96,3.792000,5
Idastidru52,15.45,3.862500,4
Chamjask73,13.83,4.610000,3
Iral74,13.62,3.405000,4
Iskadarya95,13.10,4.366667,3


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [14]:
dfnew = purchase_data[['Item ID', 'Item Name','Price']] 
item_totprice = pd.DataFrame(dfnew.groupby(["Item ID", "Item Name"]).Price.sum())
item_avgprice = pd.DataFrame(dfnew.groupby(["Item ID", "Item Name"]).Price.mean())
item_count = pd.DataFrame(dfnew.groupby(["Item ID", "Item Name"])["Item ID"].value_counts())
dfnew2 = pd.DataFrame(purchase_data[['Item ID', 'Item Name']])

a = pd.merge(dfnew2,item_count,on="Item Name",how="inner")
b = pd.merge(a,item_avgprice,on="Item Name")
c = pd.merge(b,item_totprice,on="Item Name")
d = c.rename(columns={"Price_x":"Item Price","Price_y":"Total Purchase Value", "Item ID_y": "Purchase Count","Item ID_x":"Item ID"})
e = d.sort_values("Purchase Count", ascending=False)
summarytable8 = e.drop_duplicates("Item ID")
summarytable8

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
22,92,Final Critic,13,4.614615,59.99
131,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
97,82,Nirvana,9,4.900000,44.10
0,108,"Extraction, Quickblade Of Trembling Hands",9,3.530000,31.77
126,132,Persuasion,9,3.221111,28.99
...,...,...,...,...,...
383,134,Undead Crusader,1,4.500000,4.50
776,90,Betrayer,1,2.940000,2.94
638,42,The Decapitator,1,1.750000,1.75
562,51,Endbringer,1,4.660000,4.66


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [15]:
summarytable8.sort_values("Total Purchase Value", ascending=False)

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
22,92,Final Critic,13,4.614615,59.99
131,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
97,82,Nirvana,9,4.900000,44.10
478,145,Fiery Glass Crusader,9,4.580000,41.22
558,103,Singed Scalpel,8,4.350000,34.80
...,...,...,...,...,...
652,28,"Flux, Destroyer of Due Diligence",2,1.060000,2.12
764,126,Exiled Mithril Longsword,1,2.000000,2.00
299,125,Whistling Mithril Warblade,2,1.000000,2.00
779,104,Gladiator's Glaive,1,1.930000,1.93
